In [17]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from collections import Counter
from datetime import timedelta

pd.options.display.max_columns = None

In [18]:
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
url2 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
df_usconf = pd.read_csv(url,error_bad_lines = False)
df_usdead = pd.read_csv(url2,error_bad_lines = False)

In [19]:
url = 'https://covidtracking.com/api/v1/states/daily.csv'
df_testing = pd.read_csv(url,error_bad_lines = False)
df_testing = df_testing.rename(columns = {'date':'Date','state':'Province_State'})

#filling missing starting values. May want to do this as function for all states
df_testing.loc[(df_testing['Province_State']=='CA') & (df_testing['Date']==20200304), ['positiveIncrease','negativeIncrease','totalTestResultsIncrease']] = df_testing.loc[(df_testing['Province_State']=='CA') & (df_testing['Date']==20200304), ['positive','negative','totalTestResults']].values.tolist()

df_testing['Date'] = pd.to_datetime(df_testing['Date'].astype(dtype = 'str'))

ca_testing = df_testing[['Date','Province_State','positive','negative','hospitalizedCurrently','inIcuCurrently', 'death', 'positiveIncrease','negativeIncrease','totalTestResults','totalTestResultsIncrease','deathIncrease']].loc[df_testing['Province_State'] =='CA'].copy().reset_index(drop=True)
ca_testing['Province_State'] = 'California'
ca_testing = ca_testing.sort_values(by = ['Province_State','Date']).reset_index(drop=True)

In [20]:
us_conf_daily = df_usconf.copy()
us_dead_daily = df_usdead.copy()

us_conf_daily.iloc[:,11:] = us_conf_daily.iloc[:,11:].diff(axis=1).fillna(0).astype('int')
us_dead_daily.iloc[:,12:] = us_dead_daily.iloc[:,12:].diff(axis=1).fillna(0).astype('int')

us_conf_daily = us_conf_daily.groupby(['Province_State'])[us_conf_daily.columns.to_list()[11:]].agg('sum').reset_index()
us_dead_daily = us_dead_daily.groupby(['Province_State'])[us_dead_daily.columns.to_list()[11:]].agg('sum').reset_index()

us_conf_daily = us_conf_daily.melt(id_vars = us_conf_daily.columns[:1], var_name = 'Date', value_name = 'Daily Confirmed Cases').sort_values(by = ['Province_State','Date']).reset_index(drop=True)
us_dead_daily = us_dead_daily.melt(id_vars = us_dead_daily.columns[:2], var_name = 'Date', value_name = 'Daily Dead').sort_values(by = ['Province_State','Date']).reset_index(drop=True)

us_daily = us_conf_daily.merge(us_dead_daily[['Province_State','Population','Date','Daily Dead']], on = ['Province_State','Date'], how = 'left', suffixes = (False, False))
us_daily = us_daily[['Province_State','Population','Date', 'Daily Confirmed Cases','Daily Dead']]
us_daily['Date'] = pd.to_datetime(us_daily['Date'])
us_daily = us_daily.sort_values(by = ['Province_State','Date']).reset_index(drop=True)
ca_daily = us_daily.loc[us_daily['Province_State']=='California'].copy().reset_index(drop=True)

ca_daily = ca_daily.merge(ca_testing[['Province_State', 'Date','totalTestResults','totalTestResultsIncrease','positiveIncrease','negativeIncrease','hospitalizedCurrently','inIcuCurrently']], on = ['Province_State', 'Date'], how = 'left', suffixes = (False, False))

In [21]:
ca_daily.to_csv(r'X:\AC\Documents\Datasets\CA_daily_pivot.csv')

# Data Exploration

In [16]:
most_recent = df_testing.loc[(df_testing['Province_State']=='CA')]['Date'].max()
most_recent = df_testing.loc[(df_testing['Province_State']=='CA') & (df_testing['Date']==most_recent)]
last14 = [ca_daily['Date'].max() - timedelta(days=x) for x in range(14)]
last14 = df_testing.loc[(df_testing['Province_State']=='CA') & (df_testing['Date'].isin(last14))]

current_hosp = most_recent['hospitalizedCurrently'].iloc[0].astype(int)
perc_pos = (most_recent['positive']/most_recent['totalTestResults']).iloc[0] * 100
total_tests = ca_daily['totalTestResults'][-1:]
population = ca_daily['Population'][0]
testsper1000 = total_tests/population*1000

last14tests = last14['totalTestResultsIncrease'].sum()
last14perc_pos = last14['positiveIncrease'].sum()/last14tests * 100

print('California Stats as of: ', df_testing.loc[(df_testing['Province_State']=='CA')]['Date'].max())
print('Total population in CA: %d' %population)
print('\n')
print('Total tests conducted: %d' %total_tests)
print('Total tests conducted per 1000 people in CA: %.2f' %testsper1000)
print('Percentage of total tests confirmed positive: %.2f' %perc_pos,'%')
print('Percentage of total tests confirmed negative: %.2f' %(100 - perc_pos),'%')
print(f'Currently hospitalized: {current_hosp}')
print('\n')
print('Last 14 days:')
print('Total tests conducted in last 14 days: %d' %last14tests)
print('Percentage of tests in last 14 days confirmed positive: %.2f' %last14perc_pos,'%')

California Stats as of:  2020-07-14 00:00:00
Total population in CA: 39512223


Total tests conducted: 5544365
Total tests conducted per 1000 people in CA: 140.32
Percentage of total tests confirmed positive: 5.93 %
Percentage of total tests confirmed negative: 94.07 %
Currently hospitalized: 8145


Last 14 days:
Total tests conducted in last 14 days: 1482673
Percentage of tests in last 14 days confirmed positive: 7.60 %


In [ ]:
#why are the daily confirmed cases and positive increases not lining up?
#discrepancy is from data sources. Even the cdph dashboards reporting different numbers than their tests dataset
ca_daily.tail()

# Appendix

In [ ]:
#apply date format and convert to string
.apply(lambda x: pd.datetime.strftime(x, '%m/%d/%y'))